# PROCEDURE TABLE

TODO: Create a full list of procedure_concept_ids for IDEA4RC

In [53]:
import psycopg2
import pandas as pd
import random
from datetime import datetime, timedelta

In [54]:
dbname = "idea4rc"
user = "postgres"
password = "password"
host = "localhost"
port = "5432"

conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)

cur = conn.cursor()

In [55]:
def delete_table_data(table_name):
    try:
        # Connect to your postgres DB
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        
        # Open a cursor to perform database operations
        cur = conn.cursor()
        
        # Execute a command: this deletes all data in the specified table
        cur.execute(f"DELETE FROM {table_name};")
        
        # Commit the changes
        conn.commit()
        
        # Close communication with the database
        cur.close()
        conn.close()
        
        print(f"Data from table {table_name} has been deleted.")
        
    except Exception as e:
        print(f"An error occurred: {e}")

### Generate data

In [56]:
procedure_concept_ids = [4301351, 4315400, 4217482, 4181193]

In [57]:
def generate_fake_data(num_procedures):
    
    cur.execute("SELECT person_id FROM omopcdm3.person")
    patient_ids = [row[0] for row in cur.fetchall()]
    
    procedures = []

    for procedure_id in range(1, num_procedures + 1):
        procedure = {
            'person_id': random.choice(patient_ids),
            'procedure_occurrence_id': procedure_id,
            'procedure_concept_id': random.choice(procedure_concept_ids),  # Example race concept IDs
            'procedure_date': (datetime.now() - timedelta(days=random.randint(0, 3650))).strftime('%Y-%m-%d'),  # Random date in the last 10 years
            'procedure_type_concept_id': 32879  # Example ethnicity concept IDs
        }
        procedures.append(procedure)

    return pd.DataFrame(procedures)

In [58]:
# Generate data
num_procedures = 100
df_fake_data = generate_fake_data(num_procedures)
df = df_fake_data
df.head()

,person_id,procedure_occurrence_id,procedure_concept_id,procedure_date,procedure_type_concept_id
0,44,1,4217482,2019-08-12,32879
1,41,2,4301351,2022-12-16,32879
2,89,3,4315400,2022-04-02,32879
3,64,4,4181193,2021-08-18,32879
4,79,5,4181193,2025-03-11,32879


In [59]:
delete_table_data('omopcdm3.procedure_occurrence')

sql = """
    INSERT INTO omopcdm3.procedure_occurrence (person_id, procedure_occurrence_id, procedure_concept_id, procedure_date, procedure_type_concept_id)
    VALUES (%s, %s, %s, %s, %s)
"""

cur.executemany(sql, zip(
    df.person_id, 
    df.procedure_occurrence_id, 
    df.procedure_concept_id, 
    df.procedure_date, 
    df.procedure_type_concept_id
    ))

conn.commit()

Data from table omopcdm3.procedure_occurrence has been deleted.
